In [1]:
from test_utils import *
tag = f"c0"
weight_dict = load_weight_dict_by_tag('../out/FedTest5-resnet18/2024-10-19-11:47:36/grad_lists', tag, max_round=2)
# cal_similay_by_tag('weight_lists/202405281805-avg-ss/',"server", unselect_keys=['bn', 'num_batches_tracked','downsample'], describe='202405281805-avg-ss')


/home/zhengsl/code/FL-bench/test_similar/test_utils.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight = torch.load(weight_path)


In [3]:
filter = LayerFilter(unselect_keys=['num_batches_tracked','running_mean','running_var'], all_select_keys=['layer'])

In [9]:
for i in filter(weight_dict[0][0]).keys():
    print(f"'{i}' : {weight_dict[0][0][i].shape},")

'base.layer1.0.conv1.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.0.bn1.weight' : torch.Size([64]),
'base.layer1.0.bn1.bias' : torch.Size([64]),
'base.layer1.0.conv2.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.0.bn2.weight' : torch.Size([64]),
'base.layer1.0.bn2.bias' : torch.Size([64]),
'base.layer1.1.conv1.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.1.bn1.weight' : torch.Size([64]),
'base.layer1.1.bn1.bias' : torch.Size([64]),
'base.layer1.1.conv2.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.1.bn2.weight' : torch.Size([64]),
'base.layer1.1.bn2.bias' : torch.Size([64]),
'base.layer2.0.conv1.weight' : torch.Size([128, 64, 3, 3]),
'base.layer2.0.bn1.weight' : torch.Size([128]),
'base.layer2.0.bn1.bias' : torch.Size([128]),
'base.layer2.0.conv2.weight' : torch.Size([128, 128, 3, 3]),
'base.layer2.0.bn2.weight' : torch.Size([128]),
'base.layer2.0.bn2.bias' : torch.Size([128]),
'base.layer2.0.downsample.0.weight' : torch.Size([128, 64, 1, 1]),
'base.layer2.0.downs

In [4]:
def cal_sparse(weight:Tensor):
    return 100 * (1 - torch.count_nonzero(weight) / weight.numel())
weight_0 = filter(weight_dict[0][0])
# total_size = calculate_data_size(weight_0)
for i in weight_0.keys():
    # 计算稀疏率
    print(f"{i:35} | {weight_0[i].shape} | {cal_sparse(weight_0[i]):.2f}%")
    # size = cal_memory(weight_0[i])
    # ratio = size / total_size
    # print(f"{i:35} | {size:7} bytes | {ratio * 100:.2f}% | shape: {weight_0[i].shape}")

base.layer1.0.conv1.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.0.bn1.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.bn1.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.conv2.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.0.bn2.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.bn2.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.conv1.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.1.bn1.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.bn1.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.conv2.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.1.bn2.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.

In [78]:
from functools import reduce


def cal_size(M, K, D, L):
    if isinstance(M, list):
        M = reduce(lambda x, y: x * y, M)
    w1 = K * M / L 
    w2 = D * L
    w = w1 + w2
    r = 1 - w / M
    print(f"M {M}, size: {w}, ratio: {r}, w1: {w1} {w1/w:.2f}, w2: {w2/w:.2f}")
cal_size([512, 512, 3, 3], 32, 32, 1536)

M 2359296, size: 98304.0, ratio: 0.9583333333333334, w1: 49152.0 0.50, w2: 0.50


In [1]:
def cal_K_D_L(M, K=None, D=None, L=None):
    # vector = vector.flatten()
    # M = vector.shape[0]
    if K is None and D is not None and L is not None:
        max = L // 2
        K = D * L * L // M
        if K > max:
            raise ValueError(f"Not good. K is too large, max is {max}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is None and L is not None:
        if L // K <= 2:
            raise ValueError(f"Not good. L // K is too small, min is 2, now is {L // K}")
        max = M // (4 * K)
        D = K * M // (L * L)
        if D > max:
            raise ValueError(f"Not good. D is too large, max is {max}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is not None and L is None:
        if K * D * 4 >= M:
            raise ValueError(f"Not good. K * D * 4 is too large, max is {M // 4}, now is {K * D * 4}")
        min = 2 * K
        L = int(math.sqrt(M * K // D))
        if L < min:
            raise ValueError(f"Not good. L is too small, min is {min}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is not None and L is not None:
        if K * D * 4 >= M:
            raise ValueError(f"Not good. K * D * 4 is too large, max is {M // 4}, now is {K * D * 4}")
        if L // K < 2:
            raise ValueError(f"Not good. L // K is too small, min is 2, now is {L // K}")
        r = K / L + D * L / M
    else:
        raise ValueError(f"K, D, L must have one None")
    
    if K < D:
        raise ValueError(f"Not good. K < D, now is {K} < {D}")
    return K, D, L, 1 - r



In [3]:
K, max_D, L, r = cal_K_D_L(512*512*3*3,K=32, D=32, L=256*3)
print(f"K={K}, D={max_D}, L={L}, r={r}")

K=32, D=32, L=768, r=0.9479166666666666


In [83]:
# from src.utils.compressor_utils import QuickSlideSVDCompressor
# 512, 256, 3, 3
compresser = QuickSlideSVDCompressor(64, 64, 1536, 'float32')
layername = 'base.layer4.1.conv2.weight'
v_5 = weight_dict[0][0][layername]
update_dict = compresser.update_basis_by_vector(v_5)

In [84]:
for i in range(1, 50):
    v_test = weight_dict[i][0][layername]
    a_test, e_test = compresser.compress(v_test)
    g_test = compresser.uncompress(a_test, v_test.shape)
    before = cos_similar(g_test, v_test)

    if i % 1 == 0:
        update_dict = compresser.update_basis_by_vector(v_test, update_threshold=0)
        a_test, e_test_after = compresser.compress(v_test)
        g_test = compresser.uncompress(a_test, v_test.shape)
        after = cos_similar(g_test, v_test)
        if after < before:
            if e_test_after.norm() > e_test.norm():
                print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()} e:{e_test.norm()} after e:{e_test_after.norm()}")
            else:
                print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()} e_test_after < e_test but after_similar < before_similar")
        else:
            print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()}")
    else:
        print(f"Th-{i}: before:{before:10.5f} e:{e_test.norm()} after e:{e_test_after.norm()}")

actual_D/cur_D: 36/64
Th-1: before:   0.48425, after:   0.78095 ==> Update_dict:dict_keys([0, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 21, 24, 27, 31, 32, 33, 34, 36, 37, 38, 39, 41, 42, 51, 52, 53, 54, 55, 58, 59])
actual_D/cur_D: 33/64
Th-2: before:   0.56554, after:   0.76036 ==> Update_dict:dict_keys([0, 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 31, 32, 33, 34, 35, 36, 37, 39, 43, 45, 46])
actual_D/cur_D: 25/48
Th-3: before:   0.61312, after:   0.73875 ==> Update_dict:dict_keys([0, 1, 2, 3, 4, 5, 6, 14, 15, 20, 21, 22, 23, 24, 26, 34, 35, 38, 39, 40, 47, 48, 49, 56, 60])
actual_D/cur_D: 20/36
Th-4: before:   0.65134, after:   0.69280 ==> Update_dict:dict_keys([1, 2, 3, 4, 5, 7, 9, 12, 15, 20, 21, 22, 23, 24, 26, 34, 35, 37, 40, 47])
actual_D/cur_D: 14/27
Th-5: before:   0.69481, after:   0.71810 ==> Update_dict:dict_keys([0, 1, 2, 3, 4, 38, 7, 8, 9, 39, 12, 13, 44, 25])
actual_D/cur_D: 10/20
Th-6: before:   0.72208, after:   0.73007 =

In [95]:
for i in range(1, 50):
    v_test = weight_dict[i][0]['classifier.fc1.weight']
    a_test, e_test = compresser.compress(v_test)
    g_test = compresser.uncompress(a_test, v_test.shape)
    before = cos_similar(g_test, v_test)
    print(f"{i}-th {before}")

1-th 0.3954788148403168
2-th 0.38685572147369385
3-th 0.38566067814826965
4-th 0.4078787863254547
5-th 0.39017173647880554
6-th 0.38016679883003235
7-th 0.3959686756134033
8-th 0.398908793926239
9-th 0.3915354311466217
10-th 0.4029323160648346
11-th 0.40529799461364746
12-th 0.392452597618103
13-th 0.3967442512512207
14-th 0.3904167711734772
15-th 0.3983614444732666
16-th 0.3839189410209656
17-th 0.39532017707824707
18-th 0.3997047245502472
19-th 0.3955429792404175
20-th 0.3920809030532837
21-th 0.38374629616737366
22-th 0.391897976398468
23-th 0.38946467638015747
24-th 0.3910553753376007
25-th 0.3862779438495636
26-th 0.385198712348938
27-th 0.38837939500808716
28-th 0.3965727388858795
29-th 0.3846668004989624
30-th 0.38984793424606323
31-th 0.3796626925468445
32-th 0.38867342472076416
33-th 0.3884607255458832
34-th 0.3864929974079132
35-th 0.38782989978790283
36-th 0.39541861414909363
37-th 0.3933473825454712
38-th 0.38715362548828125
39-th 0.3878777325153351
40-th 0.3889986276626587